> This exercise is courtesy of Prof. Lombardi

Mrs. Cesira manages a loundry in a small village. She has been able to survive to the economic crisys of the last years thanks to an optimal management of her activity.  
Cesira has only **two washing machines** and a electricity bill whose contract implies **a maximum power consumption of 3kW**.  
Cesira discovered that each washing cycle is composed by **two consecutive steps** with **different duration** and **different power consumption that are known**.  
On a given day Cesira analyses the load she has to wash and **she wants to complete the whole load in only 2 hours**.

|     | Washing mach. req. | Step 1 | Step 2 |
| --- | ------------------ | ------ | ------ |
|Load 1 |1 full washing machine | 1.4 kw, 20 min | 1.0 kw, 20 min|
|Load 2 |1 full washing machine |  1.6 kw, 20 min | 0.8 kw, 40 min |
|Load 3 |1 full washing machine | 1.6 kw, 20 min | 0.8 kw, 40 min |
|Load 4 |1 full washing machine | 2 kw, 20 min | 1 kw, 40 min|

Model the problem as a constraint satisfaction problem and find a solution if any.

In [1]:
# The model

from ortools.sat.python import cp_model

model = cp_model.CpModel()

# a variable s_{ij} for each load i for each step j

# Load 1
s11 = model.NewIntVar(0, 120, 's11')
s12 = model.NewIntVar(0, 120, 's12')

# Load 2
s21 = model.NewIntVar(0, 120, 's21')
s22 = model.NewIntVar(0, 120, 's22')

# Load 3
s31 = model.NewIntVar(0, 120, 's31')
s32 = model.NewIntVar(0, 120, 's32')

# Load 4
s41 = model.NewIntVar(0, 120, 's41')
s42 = model.NewIntVar(0, 120, 's42')

In [2]:
# duration constraints
model.Add(s11+20<=120)
model.Add(s12+20<=120)
model.Add(s21+20<=120)
model.Add(s22+40<=120)
model.Add(s31+20<=120)
model.Add(s32+40<=120)
model.Add(s41+20<=120)
model.Add(s42+40<=120)

# precedence constraints on the steps of each load
model.Add(s11+20 == s12)
model.Add(s21+20 == s22)
model.Add(s31+20 == s32)
model.Add(s41+20 == s42)

In [3]:
# constraints about the use of machines... two of them, does not matter which one
t11 = model.NewIntervalVar(s11, 20, s11+20, 't11')
t12 = model.NewIntervalVar(s12, 20, s12+20, 't12')
t21 = model.NewIntervalVar(s21, 20, s21+20, 't21')
t22 = model.NewIntervalVar(s22, 40, s22+40, 't22')
t31 = model.NewIntervalVar(s31, 20, s31+20, 't31')
t32 = model.NewIntervalVar(s32, 40, s32+40, 't32')
t41 = model.NewIntervalVar(s41, 20, s41+20, 't41')
t42 = model.NewIntervalVar(s42, 40, s42+40, 't42')

model.AddCumulative(
    [t11, t12, t21, t22, t31, t32, t41, t42],
    [1,   1,   1,   1,   1,   1,   1,   1],
    2)

In [4]:
# constraint about the power consumption
model.AddCumulative(
    [t11, t12, t21, t22, t31, t32, t41, t42],
    [14,  10,  16,  8,   16,  8,   20,  10],
    30)

In [5]:
# Compute a solution

solver = cp_model.CpSolver()
status = solver.Solve(model)

if status == cp_model.OPTIMAL or status == cp_model.FEASIBLE:
    print('s11 = %i' % solver.Value(s11))
    print('s12 = %i' % solver.Value(s12))
    print('s21 = %i' % solver.Value(s21))
    print('s22 = %i' % solver.Value(s22))
    print('s31 = %i' % solver.Value(s31))
    print('s32 = %i' % solver.Value(s32))
    print('s41 = %i' % solver.Value(s41))
    print('s42 = %i' % solver.Value(s42))

s11 = 0
s12 = 20
s21 = 0
s22 = 20
s31 = 40
s32 = 60
s41 = 60
s42 = 80


In [13]:
# The progam as a whole, written in a more Python-like style

# problem data
durations = [[20,20],[20,40],[20,40],[20,40]]
resource_washing_machine = [[1,1],[1,1],[1,1],[1,1]]
resource_power = [[14,10],[16,8],[16,8],[20,10]]
limit = 120
start = 0
number_of_washing_machines = 2
max_power = 30




model = cp_model.CpModel()

# create the start variables
start_variables = []
for i in range(0,4):
    for j in range(0,2):
        start_variables.append(model.NewIntVar(0, limit, 's%i%i' % (i+1, j+1)))
        
# create the interval variables
interval_variables = []
for i in range(0,4):
    for j in range(0,2):
        interval_variables.append(model.NewIntervalVar(
            start_variables[i*2+j],
            durations[i][j],
            start_variables[i*2+j]+durations[i][j],
            't%i%i' % (i+1, j+1)
        ))

# termination constraints
for i in range(0,4):
    for j in range(0,2):
        model.Add(start_variables[i*2+j]+durations[i][j]<=limit)

# precedence constraints on the steps of each load
for i in range(0,4):
        model.Add(start_variables[i*2]+durations[i][0] == start_variables[i*2+1])

# constraint about the resource of washing machines
model.AddCumulative(
    interval_variables,
    [req for load in resource_washing_machine for req in load],
    number_of_washing_machines)

# constraint about the power consumption
model.AddCumulative(
    interval_variables,
    [req for load in resource_power for req in load],
    max_power)

#-----------------------------------------------------------------------
# add a further constraint: once a load has been assigned to a washing machine,
# both the loads should be done on that machine... Cesira doesn't want to swap the loads
# create the washing machine variables
#
# The following solution adopt a technique named reification

wm_variables = []
for i in range(0,4):
    for j in range(0,2):
        wm_variables.append(model.NewIntVar(0, 1, 'wm%i%i' % (i+1, j+1)))

# impose same machine for steps of the same load
for i in range(0, len(wm_variables)-1, 2):
    model.Add(wm_variables[i]==wm_variables[i+1])
        
# create the boolean vars
wmb_variables = []
for i in range(0,len(wm_variables)):
    wmb_sub = []
    for j in range(0,i):
        wmb_sub.append(model.NewBoolVar('wmb%i_%i' % (i+1, j+1)))
    wmb_variables.append(wmb_sub)

# impose the reification
for i in range(0,len(wm_variables)):
    for j in range(0,i):
        model.Add(wm_variables[i]==wm_variables[j]).OnlyEnforceIf(wmb_variables[i][j])
for i in range(0,len(wm_variables)):
    for j in range(0,i):
        model.Add(wm_variables[i]!=wm_variables[j]).OnlyEnforceIf(wmb_variables[i][j].Not())


# impose the constraint of no-overlap
for i in range(0, len(wm_variables)):
    for j in range(0,i):
        model.Add(start_variables[j]+durations[j//2][j%2]<=start_variables[i]).OnlyEnforceIf(wmb_variables[i][j])

'''
# UNFORTUNATELY... OR-Tools DOES NOT SUPPORT reification over NoOverlap constraints
for i in range(0, len(wm_variables)):
    for j in range(0,i):
        model.AddNoOverlap([interval_variables[i], interval_variables[j]]).OnlyEnforceIf(wmb_variables[i][j])

'''
#---------------------------------------------------

solver = cp_model.CpSolver()
status = solver.Solve(model)
if status == cp_model.OPTIMAL or status == cp_model.FEASIBLE:
    for i in range(0,len(start_variables)):
        print('%s = %i' % (start_variables[i], solver.Value(start_variables[i])), end=' ')
        print(', wm = %i' % solver.Value(wm_variables[i]))       
elif status==cp_model.INFEASIBLE:
    print('INFEASIBLE')
elif status== cp_model.MODEL_INVALID:
    print('MODEL INVALID')
else:
    print('UNKONWN')

s11 = 0 , wm = 0
s12 = 20 , wm = 0
s21 = 0 , wm = 1
s22 = 20 , wm = 1
s31 = 40 , wm = 0
s32 = 60 , wm = 0
s41 = 60 , wm = 1
s42 = 80 , wm = 1
